In [2]:
from polygon import RESTClient
import pandas as pd
from dotenv import load_dotenv
import os
import datetime
import numpy as np
import statistics
load_dotenv()


True

In [3]:
portfolio_data = pd.DataFrame()

## Importing Polyon Stock Data

In [4]:
def load_portfolio_data(ticker : str, name: str, type : str, portfolio_data: pd.DataFrame,):
    client = RESTClient(os.environ['polygon_api_key'])
    aggs = client.get_aggs(
        ticker,
        1,
        "day",
        "2023-01-01",
        str(datetime.date.today()),
        limit=5000
    )
    df = pd.DataFrame(aggs).assign(ticker=ticker, name=name, type=type)
    return pd.concat([portfolio_data, df])

In [5]:
portfolio_data = load_portfolio_data(
    ticker="TSM",
    name="Taiwan Semiconductor Mfg. Co. Ltd.",
    type="stock",
    portfolio_data=portfolio_data
)

In [6]:
portfolio_data = load_portfolio_data(
    ticker="AMZN",
    name="Amazon.com Inc",
    type="stock",
    portfolio_data=portfolio_data
)

In [7]:
portfolio_data = load_portfolio_data(
    ticker="NVDA",
    name="NVIDIA Corp",
    type="stock",
    portfolio_data=portfolio_data
)

In [8]:
portfolio_data = load_portfolio_data(
    ticker="AXP",
    name="American Express Company",
    type="stock",
    portfolio_data=portfolio_data
)

In [9]:
portfolio_data = load_portfolio_data(
    ticker="CELH",
    name="Celsius Holdings, Inc.",
    type="stock",
    portfolio_data=portfolio_data
)

In [10]:
portfolio_data = load_portfolio_data(
    ticker="C:USDJPY",
    name="USD/JPY Foreign Exchange",
    type="forex",
    portfolio_data=portfolio_data
)

In [13]:
portfolio_data = load_portfolio_data(
    ticker="SPY",
    name="SPDR S&P 500 ETF Trust",
    type="etf",
    portfolio_data=portfolio_data
)

In [14]:
client = RESTClient(os.environ['polygon_api_key'])
NDX_data = pd.DataFrame(client.get_aggs(
    'I:NDX',
    1,
    "day",
    "2023-01-01",
    str(datetime.date.today()),
    limit=5000
))

In [15]:
# raw data
portfolio_data

,open,high,low,close,volume,vwap,timestamp,transactions,otc,ticker,name,type
0,75.85,75.9700,73.770,74.03,9276191.0,74.5005,1672722000000,114565,None,TSM,Taiwan Semiconductor Mfg. Co. Ltd.,stock
1,74.99,76.6100,74.470,76.32,12316386.0,75.7946,1672808400000,129761,None,TSM,Taiwan Semiconductor Mfg. Co. Ltd.,stock
2,76.00,76.4400,75.340,75.73,10039194.0,76.0024,1672894800000,101217,None,TSM,Taiwan Semiconductor Mfg. Co. Ltd.,stock
3,76.50,78.7390,76.150,78.07,13744129.0,77.6785,1672981200000,124998,None,TSM,Taiwan Semiconductor Mfg. Co. Ltd.,stock
4,80.56,81.2150,80.055,80.31,22247607.0,80.6816,1673240400000,180559,None,TSM,Taiwan Semiconductor Mfg. Co. Ltd.,stock
...,...,...,...,...,...,...,...,...,...,...,...,...
377,551.77,555.0500,551.120,554.64,40482332.0,553.5662,1720152000000,339615,None,SPY,SPDR S&P 500 ETF Trust,etf
378,555.44,556.2501,554.190,555.28,35042786.0,555.1683,1720411200000,315477,None,SPY,SPDR S&P 500 ETF Trust,etf
379,556.26,557.1800,555.520,555.82,27267633.0,556.1021,1720497600000,298706,None,SPY,SPDR S&P 500 ETF Trust,etf
380,557.07,561.6700,556.770,561.32,38356194.0,559.4119,1720584000000,339726,None,SPY,SPDR S&P 500 ETF Trust,etf


In [16]:
# Check to make sure all stock data is loaded in
portfolio_data["ticker"].unique()

array(['TSM', 'AMZN', 'NVDA', 'AXP', 'CELH', 'C:USDJPY', 'SPY'],
      dtype=object)

## Data Cleaning

In [17]:
# resetting the index
portfolio_data.reset_index(drop=True, inplace=True)

In [18]:
# dropping the otc column
portfolio_data.drop(columns=["otc"], inplace=True)

In [19]:
# converting epoch times to dates
portfolio_data["timestamp"] = pd.to_datetime(portfolio_data["timestamp"], unit="ms").dt.date
portfolio_data = portfolio_data.rename(columns={'timestamp': 'date'})
portfolio_data["date"] = pd.to_datetime(portfolio_data["date"])


In [20]:
# formatting numbers
portfolio_data = portfolio_data.round(2)
portfolio_data["volume"] = portfolio_data["volume"].astype(int)

In [21]:
# cleaning index data
NDX_data.drop(columns=["volume","vwap","transactions","otc"], inplace=True)
NDX_data = NDX_data.rename(columns={'timestamp' : 'date'})
NDX_data['date'] = pd.to_datetime(NDX_data['date'], unit='ms').dt.date
NDX_data['date'] = pd.to_datetime(NDX_data['date'])
NDX_data = NDX_data.round(2)

NDX_data

,open,high,low,close,date
0,12085.67,12156.21,12006.04,12066.27,2023-02-22
1,11979.80,12018.32,11900.84,11969.65,2023-02-24
2,12106.79,12159.64,12034.61,12057.79,2023-02-27
3,12041.75,12146.52,12021.32,12042.12,2023-02-28
4,12026.72,12054.48,11906.58,11937.48,2023-03-01
...,...,...,...,...,...
344,20224.13,20406.99,20201.50,20391.97,2024-07-05
345,20393.89,20455.38,20363.37,20439.54,2024-07-08
346,20504.17,20543.90,20395.57,20453.02,2024-07-09
347,20533.27,20690.97,20479.94,20675.38,2024-07-10


In [22]:
#checking for null values
portfolio_data.isnull().values.any()

np.False_

## Outlier Cleaning

In [23]:
# Checking for outliers in transactions and volume
portfolio_data.loc[portfolio_data["ticker"] == "C:USDJPY"].describe()

,open,high,low,close,volume,vwap,date,transactions
count,499.000000,499.000000,499.000000,499.000000,499.000000,499.000000,499,499.000000
mean,144.369739,144.882766,143.845511,144.437756,152267.092184,144.411463,2023-09-30 19:51:49.418837504,152267.092184
min,127.350000,127.880000,127.350000,127.870000,1.000000,127.880000,2023-01-01 00:00:00,1.000000
25%,136.595000,137.505000,136.285000,136.970000,109867.000000,136.805000,2023-05-13 12:00:00,109867.000000
50%,145.710000,146.220000,144.940000,145.730000,180092.000000,145.570000,2023-09-27 00:00:00,180092.000000
75%,150.095000,150.560000,149.800000,150.135000,213513.000000,150.105000,2024-02-17 00:00:00,213513.000000
max,161.580000,162.000000,161.300000,161.590000,357136.000000,161.590000,2024-07-11 00:00:00,357136.000000
std,8.498044,8.437833,8.542451,8.500011,86947.362505,8.487131,NaN,86947.362505


In [24]:
# Removing all weekend days
portfolio_data.drop(index=portfolio_data[portfolio_data["date"].dt.day_of_week>4].index, inplace=True)

# Removing all days the stock market is not open
no_holidays = portfolio_data['date'].value_counts() == 7
portfolio_data = portfolio_data[portfolio_data['date'].isin(no_holidays[no_holidays].index)]


## Aggregations

Calculating how many shares of what to buy (buying based off vwap and partial shares are allowed)
- 40% SPY
- 10% USDJPY
- 10% AMZN
- 10% TSM
- 10% CELH
- 10% AXP
- 10% NVDA

In [25]:
SPY_shares = 40000/portfolio_data[portfolio_data["ticker"] == "SPY"]["vwap"].iat[0]
USDJPY_shares = 10000/portfolio_data[portfolio_data["ticker"]=="C:USDJPY"]["vwap"].iat[0]
AMZN_shares = 10000/portfolio_data[portfolio_data["ticker"]=="AMZN"]["vwap"].iat[0]
TSM_shares = 10000/portfolio_data[portfolio_data["ticker"]=="TSM"]["vwap"].iat[0]
CELH_shares = 10000/portfolio_data[portfolio_data["ticker"]=="CELH"]["vwap"].iat[0]
AXP_shares = 10000/portfolio_data[portfolio_data["ticker"]=="AXP"]["vwap"].iat[0]
NVDA_shares = 10000/portfolio_data[portfolio_data["ticker"]=="NVDA"]["vwap"].iat[0]

Creating dataframes for each investment

In [26]:
cols = ['date','open','high','low','close','volume','vwap','transactions']
SPY_etf_data = portfolio_data[portfolio_data["ticker"] == "SPY"][cols].reset_index(drop=True)
USDJPY_forex_data = portfolio_data[portfolio_data["ticker"] == "C:USDJPY"][cols].reset_index(drop=True)
AMZN_stock_data = portfolio_data[portfolio_data["ticker"] == "AMZN"][cols].reset_index(drop=True)
TSM_stock_data = portfolio_data[portfolio_data["ticker"] == "TSM"][cols].reset_index(drop=True)
AXP_stock_data = portfolio_data[portfolio_data["ticker"] == "AXP"][cols].reset_index(drop=True)
CELH_stock_data = portfolio_data[portfolio_data["ticker"] == "CELH"][cols].reset_index(drop=True)
NVDA_stock_data = portfolio_data[portfolio_data["ticker"] == "NVDA"][cols].reset_index(drop=True)

In [27]:

portfolio_tickers = pd.concat([
    portfolio_data[portfolio_data["ticker"] == "SPY"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "C:USDJPY"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "AMZN"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "TSM"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "AXP"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "CELH"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "NVDA"][['ticker','name','type']].head(1)
]).reset_index(drop=True)

portfolio_tickers

,ticker,name,type
0,SPY,SPDR S&P 500 ETF Trust,etf
1,C:USDJPY,USD/JPY Foreign Exchange,forex
2,AMZN,Amazon.com Inc,stock
3,TSM,Taiwan Semiconductor Mfg. Co. Ltd.,stock
4,AXP,American Express Company,stock
5,CELH,"Celsius Holdings, Inc.",stock
6,NVDA,NVIDIA Corp,stock


Performing Stock/ETF/Forex aggregations

In [28]:
# Market Aggregations
NDX_data['average'] = ((NDX_data['high'] + NDX_data['low'])/2).round(2)
NDX_data['return'] = (NDX_data['average'] - NDX_data['average'].iat[0])/NDX_data['average'].iat[0]

NDX_data

,open,high,low,close,date,average,return
0,12085.67,12156.21,12006.04,12066.27,2023-02-22,12081.12,0.000000
1,11979.80,12018.32,11900.84,11969.65,2023-02-24,11959.58,-0.010060
2,12106.79,12159.64,12034.61,12057.79,2023-02-27,12097.12,0.001324
3,12041.75,12146.52,12021.32,12042.12,2023-02-28,12083.92,0.000232
4,12026.72,12054.48,11906.58,11937.48,2023-03-01,11980.53,-0.008326
...,...,...,...,...,...,...,...
344,20224.13,20406.99,20201.50,20391.97,2024-07-05,20304.25,0.680660
345,20393.89,20455.38,20363.37,20439.54,2024-07-08,20409.38,0.689362
346,20504.17,20543.90,20395.57,20453.02,2024-07-09,20469.74,0.694358
347,20533.27,20690.97,20479.94,20675.38,2024-07-10,20585.46,0.703936


In [39]:
def agg_stock(stock_data: pd.DataFrame, shares: float, rf: float):
    stock_data['value'] = stock_data['vwap'] * shares.round(2)
    stock_data['cumulative return'] = (stock_data['value'] - stock_data['value'].iat[0])/stock_data['value'].iat[0]
    stock_data['volatility'] = stock_data['vwap'].rolling(len(stock_data),min_periods=1).std()
    stock_data['sharpe'] = (stock_data['cumulative return'] - rf)/(stock_data['cumulative return'].rolling(len(stock_data), min_periods=1).std())
    stock_data['10 day avg'] = stock_data['vwap'].rolling(window=10,min_periods=1).mean()
    stock_data['100 day avg'] = stock_data['vwap'].rolling(window=100,min_periods=1).mean()
    stock_data['beta'] = stock_data['cumulative return'].rolling(len(stock_data),min_periods=1).cov(NDX_data['return'])/stock_data['cumulative return'].rolling(len(stock_data),min_periods=1).var()

    stock_data.fillna(0,inplace=True)

In [46]:
#interest rate of a 3 year bond
rf = 0.0546

agg_stock(SPY_etf_data, SPY_shares, rf)
agg_stock(AMZN_stock_data, AMZN_shares, rf)
agg_stock(TSM_stock_data, TSM_shares, rf)
agg_stock(AXP_stock_data, AXP_shares, rf)
agg_stock(CELH_stock_data, CELH_shares, rf)
agg_stock(NVDA_stock_data, NVDA_shares, rf)

USDJPY_forex_data['value'] = (USDJPY_forex_data['vwap'] * USDJPY_shares).round(2)
USDJPY_forex_data['cumulative return'] = (USDJPY_forex_data['value'] - USDJPY_forex_data['value'].iat[0])/USDJPY_forex_data['value'].iat[0]
USDJPY_forex_data['percent change'] = USDJPY_forex_data['value'].pct_change()
USDJPY_forex_data.fillna(0, inplace=True)

AXP_stock_data

,date,open,high,low,close,volume,vwap,transactions,value,cumulative return,volatility,sharpe,10 day avg,100 day avg,beta
0,2023-01-03,149.54,150.45,145.73,147.12,2762338,147.43,37709,10000.1769,0.000000,0.000000,0.000000,147.430000,147.430000,0.000000
1,2023-01-04,148.56,151.96,148.56,150.54,2620388,150.60,36930,10215.1980,0.021502,2.241528,-2.176942,149.015000,149.015000,-0.467884
2,2023-01-05,147.16,148.72,146.02,146.43,2505858,146.79,42670,9956.7657,-0.004341,2.040204,-4.259220,148.273333,148.273333,-0.449319
3,2023-01-06,147.74,150.43,145.48,150.17,3077593,149.36,42792,10131.0888,0.013091,1.752189,-3.492590,148.545000,148.545000,-0.365103
4,2023-01-09,151.35,152.44,150.10,150.40,3219406,150.88,41307,10234.1904,0.023401,1.842029,-2.497072,149.012000,149.012000,-0.373578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,2024-07-05,235.71,236.70,233.56,235.63,2276542,235.39,50466,15966.5037,0.596622,30.089076,2.655792,232.354000,227.672700,0.565556
378,2024-07-08,236.23,239.50,233.76,234.51,2352443,235.35,46111,15963.7905,0.596351,30.172720,2.647104,232.854000,227.899200,0.562424
379,2024-07-09,234.97,239.20,234.12,237.75,1825409,236.85,41549,16065.5355,0.606525,30.262053,2.688857,233.324000,228.176300,0.559109
380,2024-07-10,236.57,239.45,235.62,238.93,1859309,237.79,43596,16129.2957,0.612901,30.354435,2.711641,234.072000,228.451600,0.555711


Performing portfolio aggregations

In [54]:
portfolio_aggs = pd.DataFrame(SPY_etf_data['date'])
portfolio_aggs['value'] = (
    SPY_etf_data['value'] +
    AMZN_stock_data['value'] +
    TSM_stock_data['value'] +
    AXP_stock_data['value'] +
    CELH_stock_data['value'] +
    NVDA_stock_data['value'] +
    USDJPY_forex_data['value']
).round(2)
portfolio_aggs['cumulative return'] = (portfolio_aggs['value'] - portfolio_aggs['value'].iat[0])/portfolio_aggs['value'].iat[0]

portfolio_aggs['annualized return'] = (
    (1 + portfolio_aggs['cumulative return']).pow(365/(portfolio_aggs['date'] - portfolio_aggs['date'].iat[0]).dt.days) - 1
)

less_than_1_year = (portfolio_aggs['date'] - portfolio_aggs['date'].iat[0]).dt.days < 365
# Global Investment Performance Standards dictate that returns of portfolios or composites for periods of less than one year may not be annualized
portfolio_aggs.loc[less_than_1_year, 'annualized return'] = portfolio_aggs['cumulative return']

# weighted average returns
portfolio_aggs['volatility'] = (
    0.4 * SPY_etf_data['cumulative return'] +
    0.1 * TSM_stock_data['cumulative return'] +
    0.1 * AMZN_stock_data['cumulative return'] +
    0.1 * AXP_stock_data['cumulative return'] +
    0.1 * CELH_stock_data['cumulative return'] +
    0.1 * NVDA_stock_data['cumulative return'] +
    0.1 * USDJPY_forex_data['cumulative return']
    )

portfolio_aggs['sharpe'] = (portfolio_aggs['cumulative return'] - rf)/(portfolio_aggs['cumulative return'].rolling(len(portfolio_aggs),min_periods=1).std())
portfolio_aggs.fillna(0,inplace=True)

portfolio_aggs

,date,value,cumulative return,annualized return,volatility,sharpe
0,2023-01-03,100001.01,0.000000,0.000000,0.000000,0.000000
1,2023-01-04,100822.31,0.008213,0.008213,0.008213,-7.987569
2,2023-01-05,100105.05,0.001040,0.001040,0.001040,-11.977313
3,2023-01-06,101044.72,0.010437,0.010437,0.010437,-8.523502
4,2023-01-09,103386.32,0.033853,0.033853,0.033853,-1.515061
...,...,...,...,...,...,...
377,2024-07-05,240031.15,1.400287,0.789848,1.400300,3.511940
378,2024-07-08,241368.85,1.413664,0.790763,1.413677,3.528493
379,2024-07-09,242539.55,1.425371,0.794599,1.425384,3.540247
380,2024-07-10,246301.80,1.462993,0.810979,1.463007,3.616708


## Exporting to SQL Server

In [63]:
portfolio_aggs.to_csv('portfolio_aggs.csv', index=False)
NDX_data.to_csv('NDX_data.csv', index=False)

In [61]:
portfolio_tickers.to_csv('portfolio_tickers.csv', index=False)

In [62]:
AMZN_stock_data.to_csv('AMZN_stock_data.csv',index=False)
AXP_stock_data.to_csv('AXP_stock_data.csv',index=False)
CELH_stock_data.to_csv('CELH_stock_data.csv',index=False)
NVDA_stock_data.to_csv('NVDA_stock_data.csv',index=False)
TSM_stock_data.to_csv('TSM_stock_data.csv',index=False)
SPY_etf_data.to_csv('SPY_etf_data.csv',index=False)
USDJPY_forex_data.to_csv('USDJPY_forex_data.csv',index=False)